In [1]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [ ]:

        df1=filtered_df.groupby(['Launch Site','class']).size().reset_index(name='class count')
        fig = px.pie(df1, values='class count', names='class', title=f"Total Success Launches for site {entered_site}")
        return fig


In [2]:

filtered_df = spacex_df[spacex_df['Launch Site'] == 'CCAFS LC-40']

In [10]:
filtered_df.groupby(['Launch Site', 'class']).size().reset_index()

,Launch Site,class,0
0,CCAFS LC-40,0,19
1,CCAFS LC-40,1,7


In [ ]:
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                html.Div(dcc.Dropdown(id='site-dropdown',
                                 options=[
                                     {'label':'All Sites','value':'ALL'},
                                     {'label':'CCAFS LC-40','value':'CCAFS LC-40'},
                                     {'label':'CCAFS SLC-40','value':'CCAFS SLC-40'},
                                     {'label':'KSC LC-39A','value':'KSC LC-39A'},
                                     {'label':'VAFB SLC-4E','value':'VAFB SLC-4E'}
                                 ], value = 'ALL',
                                 placeholder = 'Select a launch Site here',
                                 searchable = True)),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',min=0,max=10000,step=1000,value=[min_payload,max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id = 'success-pie-chart',component_property = 'figure'),
              Input(component_id = 'site-dropdown',component_property = 'value'))

def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site=='ALL':
        fig=px.pie(spacex_df,values='class',
        names='Launch Site',
        title='Success Count for all launch sites')
        return fig
    else:
        df_site_filtered=spacex_df[spacex_df['Launch Site']==site]
        filtered_df=spacex_df[spacex_df['Launch Site']==entered_site]
        df1=filtered_df.groupby(['Launch Site','class']).size().reset_index(name='class count')
        fig = px.pie(df1, values='class count', names='class', title=f"Total Success Launches for site {entered_site}")
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output


@app.callback(Output(component_id='success-payload-scatter-chart',component_property='figure'),
                [Input(component_id='site-dropdown',component_property='value'),
                Input(component_id='payload-slider',component_property='value')])
def scatter(site,payload):
    low, high = (payload[0],payload[1])
    mask=spacex_df[spacex_df['Payload Mass (kg)'].between(low,high)]
    if site=='ALL':
        fig=px.scatter(mask,x='Payload Mass (kg)',y='class',color='Booster Version Category',title='Success count on Payload mass for all sites')
        return fig
    else:
        mask_filtered=mask[mask['Launch Site']==site]
        fig=px.scatter(mask_filtered,x='Payload Mass (kg)',y='class',color='Booster Version Category',title='Success count on Payload Mass for' + site)
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Sep/2021 20:14:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Sep/2021 20:14:20] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [18/Sep/2021 20:14:20] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [18/Sep/2021 20:14:20] "GET /_dash-component-suites/dash_core_components/async-slider.v1_16_0m1617903285.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Sep/2021 20:14:21] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Sep/2021 20:14:21] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Sep/2021 20:14:22] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\python39\lib\site-packages\flask\app.py", line 2070, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\python39\lib\site-packages\flask\app.py", line 1515, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\python39\lib\site-packages\flask\app.py", line 1513, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\python39\lib\site-packages\flask\app.py", line 1499, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "C:\Users\Asus\AppData\Roaming\Python\Python39\site-packages\dash\dash.py", line 1079, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "C:\Users\Asus\AppData\Roaming\Python\Python39\site-packages\dash\dash.py", line 1010, in add_context
    output_value = func(*args, **kwargs)  # %% callback invoked %%
  File "<ipython-input-2-425987e227b7>", l

127.0.0.1 - - [18/Sep/2021 20:14:22] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [18/Sep/2021 20:14:34] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Sep/2021 20:14:34] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_9_1m1625147221.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Sep/2021 20:14:34] "GET /_dash-component-suites/dash_renderer/react@16.v1_9_1m1625147221.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Sep/2021 20:14:34] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_9_1m1625147221.8.7.min.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Sep/2021 20:14:34] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_9_1m1625147221.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Sep/2021 20:14:34] "GET /_dash-component-suites/dash_core_components/dash_core_components-shared.v1_16_0m1625147222.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Sep/2021 20:14:34] "GET /_dash-component-suites/dash_html_components/dash_html_components.v1_1_3m1625147221.min.js HTTP/1.1" 200 -
127.0.

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\python39\lib\site-packages\flask\app.py", line 2070, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\python39\lib\site-packages\flask\app.py", line 1515, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\python39\lib\site-packages\flask\app.py", line 1513, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\python39\lib\site-packages\flask\app.py", line 1499, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "C:\Users\Asus\AppData\Roaming\Python\Python39\site-packages\dash\dash.py", line 1079, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "C:\Users\Asus\AppData\Roaming\Python\Python39\site-packages\dash\dash.py", line 1010, in add_context
    output_value = func(*args, **kwargs)  # %% callback invoked %%
  File "<ipython-input-2-425987e227b7>", l

127.0.0.1 - - [18/Sep/2021 20:14:37] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [18/Sep/2021 20:14:37] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Sep/2021 20:14:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Sep/2021 20:14:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Sep/2021 20:14:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Sep/2021 20:14:55] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Sep/2021 20:14:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Sep/2021 20:14:59] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Sep/2021 20:15:03] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Sep/2021 20:15:05] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Sep/2021 20:15:07] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\python39\lib\site-packages\flask\app.py", line 2070, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\python39\lib\site-packages\flask\app.py", line 1515, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\python39\lib\site-packages\flask\app.py", line 1513, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\python39\lib\site-packages\flask\app.py", line 1499, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "C:\Users\Asus\AppData\Roaming\Python\Python39\site-packages\dash\dash.py", line 1079, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "C:\Users\Asus\AppData\Roaming\Python\Python39\site-packages\dash\dash.py", line 1010, in add_context
    output_value = func(*args, **kwargs)  # %% callback invoked %%
  File "<ipython-input-2-425987e227b7>", l

127.0.0.1 - - [18/Sep/2021 20:15:11] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [18/Sep/2021 20:15:11] "POST /_dash-update-component HTTP/1.1" 200 -
